In [7]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [2]:
class Memory(nn.Module):
    def __init__(self, N, M):
        self.memory = torch.zeros(N, M)

    
         

In [24]:
class AddressingMechanism:
    def __init__(self, w, M, k, beta, g, s, gamma, shifts=[-1, 0, 1]):
        self.w = w
        self.M = M
        self.k = k
        self.beta = beta
        self.g = g
        self.s = s
        self.gamma = gamma

        self.N = self.M.shape[0]
        self.shifts = shifts

    def content_addressing(self):
        sim = F.cosine_similarity(self.k.unsqueeze(1), self.M.unsqueeze(0), dim=-1)
        self.wc = F.softmax(self.beta.unsqueeze(1)*sim, dim=0)

    def interpolation(self):
        self.wg = self.g*self.wc + (1 - self.g)*self.w
        

    def shift(self):
        w_tilde = torch.zeros(self.N)
        for i in range(self.N)
            for j in range(self.N):
                if (i - j) % self.N in self.shifts:
                    w_tilde[i] += self.wg[j]*self.s[]

    def sharpening(self):
        w_tilde = torch.pow(self.w_tilde, self.gamma)
        self.wt = w_tilde/w_tilde.sum()    


In [67]:
import numpy as np

shifts = torch.tensor([-1, 0, 1])
(shifts==0).nonzero(as_tuple=False).squeeze()

tensor(1)

In [37]:
sim = F.cosine_similarity(k, M)
print(F.softmax(sim, dim=0)[0])

sim_s = F.cosine_similarity(k, M)
F.softmax(sim_s, dim=0)[0]


tensor(0.0568)


tensor(0.0568)

In [32]:
torch.exp(sim[0])/torch.exp(sim).sum()

tensor(0.0568)

In [23]:
(M[0]*k).sum()/(torch.norm(M[0])*torch.norm(k))

tensor(0.8503)

In [18]:
(M[0]*k).sum()

tensor(2.4707)

In [70]:
def content_addressing(k, M, beta):
    sim = F.cosine_similarity(k, M)
    wc = F.softmax(beta*sim, dim=0)
    return wc


In [119]:
wc = torch.zeros(100, 20)
for i in range(k.shape[0]):
    wc[i] = F.cosine_similarity(k[i], M)

In [120]:
wc2 = F.cosine_similarity(k.unsqueeze(1), M.unsqueeze(0), dim=-1)

In [125]:
F.softmax(wc2, dim=0).sum(dim=0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000])

In [129]:
a = torch.rand(2, 5)
b = torch.tensor([1, 6])

In [134]:
b.unsqueeze(1)*a

tensor([[0.8539, 0.9458, 0.2876, 0.7212, 0.5269],
        [4.7286, 4.5559, 2.6117, 2.0051, 1.5541]])

In [133]:
a

tensor([[0.8539, 0.9458, 0.2876, 0.7212, 0.5269],
        [0.7881, 0.7593, 0.4353, 0.3342, 0.2590]])

In [254]:
k = torch.rand(100, 10)
M = torch.rand(100, 20, 10)
beta = torch.rand(100)
g = torch.rand(100)
w = torch.rand(100, 20)
gamma = torch.rand(100)
s = torch.rand(100, 3)
shifts = torch.tensor([-1, 0, 1])


def content_addressing(k, M, beta):
    sim = F.cosine_similarity(k.unsqueeze(1), M.unsqueeze(0), dim=-1)
    wc = F.softmax(beta.unsqueeze(1)*sim, dim=0)
    wc = wc.squeeze(0)
    return wc

def interpolation(g, wc, w):
    g = g.unsqueeze(1)
    wg = g*wc + (1 - g)*w
    return wg

def shift(wg, s, shifts):
    N = wg.shape[1]
    w_tilde = torch.zeros_like(wg)
    for i in range(N):
        for j in range(N):
            if (i - j) % N in shifts:
                w_tilde[:, i] = wg[:, j]*s[:, (shifts==((i - j) % N)).nonzero().squeeze()]
    return w_tilde


def sharpening(w_tilde, gamma):
    w_tilde = torch.pow(w_tilde, gamma.unsqueeze(1))
    wt = w_tilde/w_tilde.sum()   
    return wt 



In [261]:
wc = content_addressing(k, M, beta)
wg = interpolation(g, wc, w)
w_tilde = shift(wg, s, shifts)
wt = sharpening(w_tilde, gamma)

torch.einsum('ij, ijk->ik', w, M).shape

torch.Size([100, 10])

In [241]:
(shifts == 0).nonzero()

tensor([[1]])

In [256]:
M.shape

torch.Size([100, 20, 10])